In [1]:
import pandas as pd
import numpy as np
import os
import yaml
from easydict import EasyDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import normalized_mutual_info_score as NMI

from z_common import config_to_execute
from data_aug.gaussian_blur import GaussianBlur
from clustering import run_clustering
from dataset import get_datasets
from simclr import SimCLR
from log_functions import *

from models.baseline_encoder import Encoder
from models.alexnet_simclr import AlexSimCLR
from models.resnet_simclr import ResNetSimCLR
from loss.nt_xent import NTXentLoss
from functions import *

/home/syamagami/anaconda3/envs/generic/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Please install apex for mixed precision training from: https://github.com/NVIDIA/apex


In [3]:
N_CUDA = 1
PARENT = 'Office31'
DATA_DOMAIN = 'dslr_webcam'
# PARENT = 'Digit'
# DATA_DOMAIN = 'svhn_synth_mnist_mnistm'

log_dir_opt = f''

config, CONFIG_FILE, log_dir = config_to_execute(DATA_DOMAIN, PARENT, N_CUDA, log_dir_opt)
config = yaml.load(open(CONFIG_FILE, "r"), Loader=yaml.FullLoader)
config = EasyDict(config)

# args
config.config = CONFIG_FILE
config.log_dir = log_dir
config.num_laps = 2
config.spread_message = ''
config.cuda_dir = 2
# manually
config.device = "cuda"
config.target_dsets_name = "_".join(np.array(config.dataset.dset_taples, dtype=object)[:,0])  # dset名のリスト
config.checkpoints_dir = os.path.join(config.log_dir, 'checkpoints')
config.lap = 1

In [3]:
all_filenames = get_all_filenames(config)
edls = np.random.randint(0, 2, size=(len(all_filenames), config.model.out_dim))


In [12]:
tens = torch.tensor([
    [0.1540, 0.2633, 0.2743, 0.2106, 0.1614, 0.4289],
    [0.1451, 0.2510, 0.1784, 0.3647, 0.1584, 0.1981],
    [0.1309, 0.1992, 0.1763, 0.5405, 0.2941, 0.1851],
    [0.1389, 0.1890, 0.1584, 0.5859, 0.2454, 0.2257],
    [0.2326, 0.3690, 0.3542, 0.5884, 0.3860, 0.5289],
    [0.1226, 0.2057, 0.2105, 0.5414, 0.2007, 0.3194]
    ], dtype=torch.float32)

In [13]:
torch.round(tens)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 0.]])

In [5]:
ed_feats = np.loadtxt("./dslr_webcam_ed_feats.csv", delimiter=",")
ed_feats

array([[ 0.68366182,  0.97465956,  0.24804604, ...,  0.64804476,
         0.73727608,  1.02737713],
       [ 0.28964758,  0.8780452 ,  0.39023322, ...,  0.6203447 ,
         0.83915722,  0.91162282],
       [ 0.37241447,  0.96449316, -0.0473991 , ...,  0.83186758,
         0.6270299 ,  0.92077935],
       ...,
       [ 0.77629513,  0.65654004, -0.27979779, ...,  0.77521229,
         0.88972056,  1.04037678],
       [ 0.5391078 ,  0.9754343 , -0.07070674, ...,  0.59811008,
         0.26082897,  1.02409172],
       [ 0.8143878 ,  1.07154799,  0.00901337, ...,  0.78825128,
         0.97075415,  1.15027678]])

In [10]:
import math
import random
import numpy as np
[random.random() * math.pi for _ in range(10)]
np.random.normal(0, 1, size=10)

array([ 0.71096135, -0.34807661,  0.27453618,  0.44932769,  2.45043913,
        0.10925905, -0.17747518, -0.31850479, -0.64726114, -1.71179602])